In [13]:
import torch
import pandas as pd
import numpy as np
import random
import os
import scipy.special

## Load Data & Pre-Processing

In [14]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.head()

,ID,age,gender,tenure,frequent,payment_interval,subscription_type,contract_length,after_interaction,support_needs
0,TRAIN_00000,54.0,F,47.0,22.0,8.0,member,90,25.0,0
1,TRAIN_00001,30.0,M,16.0,15.0,5.0,vip,360,23.0,0
2,TRAIN_00002,29.0,M,8.0,30.0,21.0,plus,30,21.0,0
3,TRAIN_00003,38.0,F,38.0,23.0,10.0,vip,90,6.0,0
4,TRAIN_00004,25.0,F,52.0,3.0,17.0,member,30,1.0,2


In [15]:
train['gender'] = train['gender'].map({'F':0, "M": 1})
test['gender'] = test['gender'].map({'F':0, "M": 1})

train['subscription_type'] = train['subscription_type'].map({'member' : 0, 'plus' : 1, 'vip' : 2})
test['subscription_type'] = test['subscription_type'].map({'member' : 0, 'plus' : 1, 'vip' : 2})

In [36]:
x_train = train.drop(columns =['ID', 'support_needs'])
x_test = test.drop(columns = ['ID'])

y_train = train['support_needs']
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_train_onehot = torch.nn.functional.one_hot(y_train_tensor, num_classes=3)
print(y_train_onehot)

tensor([[1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        ...,
        [1, 0, 0],
        [1, 0, 0],
        [0, 0, 1]])


In [17]:
x_train.head()

,age,gender,tenure,frequent,payment_interval,subscription_type,contract_length,after_interaction
0,54.0,0,47.0,22.0,8.0,0,90,25.0
1,30.0,1,16.0,15.0,5.0,2,360,23.0
2,29.0,1,8.0,30.0,21.0,1,30,21.0
3,38.0,0,38.0,23.0,10.0,2,90,6.0
4,25.0,0,52.0,3.0,17.0,0,30,1.0


In [32]:
categorical_cols = ['gender', 'subscription_type']
numerical_cols = [col for col in x_train.columns if col not in categorical_cols]


# pandas DataFrame -> torch.Tensor 변환
x_train_tensor = torch.tensor(x_train[numerical_cols].values, dtype=torch.float32)
x_test_tensor  = torch.tensor(x_test[numerical_cols].values, dtype=torch.float32)

# 평균, 표준편차 계산 (학습 데이터 기준)
mean = x_train_tensor.mean(dim=0, keepdim=True)
std = x_train_tensor.std(dim=0, keepdim=True)

# 표준화
x_train_tensor = (x_train_tensor - mean) / std
x_test_tensor  = (x_test_tensor - mean) / std


## Train

In [33]:
#neural network class definition
class neuralNetwork:

    #initialize the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        #set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        #가중치 행렬 wih(input_hidden)와 who(hidden_output)
        #배열 내 가중치는 w_i_j로 표기. 노드 i에서 다음 계층의 노드 j로 연결됨을 의미
        #w11 w21
        #w12 w22

        self.wih = np.random.normal(0.0, pow(self.hnodes, - 0.5),(self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, - 0.5),(self.onodes, self.hnodes))

        #learning rate
        self.lr = learningrate

        #expit()는 시그모이드 함수
        self.activation_function = lambda x: scipy.special.expit(x)

        pass

    #신경망 학습시키기
    def train(self, inputs_list, targets_list):
      #입력 리스트를 2차원의 행렬로 변환
      inputs = np.array(inputs_list, ndmin=2).T
      targets = np.array(targets_list, ndmin=2).T
# 순전파
      #은닉 계층으로 들어오는 신호를 계산
      hidden_inputs = np.dot(self.wih, inputs)
      #은닉 계층에서 나가는 신호를 계산
      hidden_outputs = self.activation_function(hidden_inputs)

      #최종 출력 계층으로 들어오는 신호를 계산
      final_inputs = np.dot(self.who, hidden_outputs)
      #최종 출력 계층에서 나가는 신호를 계산
      final_outputs = self.activation_function(final_inputs)

# 역전파
      #출력 계층의 오차는 (실제 값 - 계산 값)
      output_errors = targets - final_outputs
      #은닉 계층의 오차는 가중치에 의해 나뉜 계층의 오차들을 재조합해 계산
      hidden_errors = np.dot(self.who.T, output_errors)
# 가중치 업데이트
      #은닉 계층과 출력 계층 간의 가중치 업데이트
      self.who += self.lr*np.dot((output_errors*final_outputs*(1.0-final_outputs)), np.transpose(hidden_outputs))

      #입력 계층과 은닉 계층 간의 가중치 업데이트
      self.wih += self.lr*np.dot((hidden_errors*hidden_outputs*(1.0-hidden_outputs)), np.transpose(inputs))
      pass


    #query the neural network
    def query(self, inputs_list):
      #입력 리스트를 2차원 행렬로 변환
      inputs = np.array(inputs_list, ndmin=2).T
      #은닉 계층으로 들어오는 신호를 계산
      #x_hidden = w_hidden x I (x는 가중치 행렬, I는 입력 값들의 행렬)
      #입력 계층과 은닉 계층 사이의 가중치 행렬은 입력 핼렬과 조합됨
      hidden_inputs = np.dot(self.wih, inputs)
      #은닉 계층에서 나가는 신호를 계산
      hidden_outputs = self.activation_function(hidden_inputs)
      #최종 출력 계층으로 들어오는 신호 계산
      final_inputs = np.dot(self.who, hidden_outputs)
      #최종 출력 계층에서 나가는 신호 계산
      final_outputs = self.activation_function(final_inputs)

      return final_outputs

In [34]:
#입력, 은닉, 출력 노드의 수
input_nodes = 8
hidden_nodes = 10
output_nodes = 3

#학습률은 0.3
learning_rate = 0.1

#인경망의 인스턴스(instance)를 생성
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [37]:
epochs = 10
for e in range(epochs):
    for i in range(len(x_train_tensor)):
        # torch.Tensor -> numpy 변환
        inputs = x_train_tensor[i].numpy()
        targets = y_train_onehot[i].numpy()   # y_train도 torch.Tensor라면 변환 필요
        n.train(inputs, targets)

    if e % 10 == 0:
        print(f"epoch: {e}")

ValueError: shapes (10,8) and (6,1) not aligned: 8 (dim 1) != 6 (dim 0)

## Prediction

In [30]:
submission_csv = pd.read_csv('sample_submission.csv')
submission=submission_csv['support_needs']
print(submission[0])

0


In [31]:
import numpy as np
import pandas as pd

preds = []
for i in range(len(x_test)):
    inputs = x_test.iloc[i].values
    outputs = n.query(inputs)           # shape (3,1)
    label = np.argmax(outputs)          # 가장 큰 값의 index = 클래스 예측
    preds.append(label)

# DataFrame에 예측값 저장
submission = pd.read_csv('sample_submission.csv')
submission['support_needs'] = preds
submission.to_csv('baseline_submit.csv', index=False)

print("저장 완료: baseline_submit.csv")


저장 완료: baseline_submit.csv


## Submission